<a href="https://colab.research.google.com/github/minthammock-development/Technical-Assignments/blob/main/RingCentral_Workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import pandas as pd
import numpy as np

In [250]:
# Custom Functions
def breakColumns(df, columns, conditionRevenue = []):
  for column in columns:
    entries = df[column].value_counts().index
    for entry in entries:
      df[entry] = [1 if x == entry else 0 for x in df[column]]
      if column in conditionRevenue:
        df[f'{entry} Revenue'] = [df['Forecasted Annual Revenue'][i] if (df[column][i] == entry and df['Stage'][i] == '7. Closed Won') else 0 for i in range(len(df))]
        df[f'{entry} Lost Revenue'] = [df['Forecasted Annual Revenue'][i] if (df[column][i] == entry and df['Stage'][i] == '0. Downgraded') else 0 for i in range(len(df))]
        df[f'{entry} Closed Sale'] = [1 if (df[column][i] == entry and df['Stage'][i] == '7. Closed Won') else 0 for i in range(len(df))]


In [251]:
dfPartners = pd.read_excel('/content/drive/MyDrive/Reporting Working Example-S&D Details-Partner Operations Analyst.xlsx',sheet_name='Partners')
dfOpp = pd.read_excel('/content/drive/MyDrive/Reporting Working Example-S&D Details-Partner Operations Analyst.xlsx', sheet_name='Opportunities')

In [252]:
display(dfPartners.loc[dfPartners['Partner ID'].duplicated() == True])
dfPartners = dfPartners.drop(dfPartners.loc[dfPartners['Partner ID'].duplicated() == True].index)

,Partner Name,Partner ID,Channel Manager
186,Partner 536,44764,Channel Manager 6
271,Partner 543,56304,Channel Manager 6
298,Partner 490,58007,Channel Manager 4
308,Partner 599,59128,Channel Manager 7
454,Partner 556,74611,Channel Manager 6
456,Partner 533,75119,Channel Manager 5
457,Partner 557,75119,Channel Manager 6
493,Partner 617,79257,Channel Manager 7
498,Partner 618,79265,Channel Manager 7
514,Partner 566,79349,Channel Manager 6


In [253]:
display(dfPartners.head())
dfPartners.info()
dfPartners['Partner ID'].value_counts()

,Partner Name,Partner ID,Channel Manager
0,Partner 592,478,Channel Manager 7
1,Partner 172,530,Channel Manager 3
2,Partner 173,1519,Channel Manager 3
3,Partner 174,2539,Channel Manager 3
4,Partner 175,2564,Channel Manager 3


<class 'pandas.core.frame.DataFrame'>
Int64Index: 622 entries, 0 to 635
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Partner Name     622 non-null    object
 1   Partner ID       622 non-null    int64 
 2   Channel Manager  622 non-null    object
dtypes: int64(1), object(2)
memory usage: 19.4+ KB


79351    1
98652    1
91161    1
60771    1
79744    1
        ..
90159    1
96192    1
44713    1
17064    1
81443    1
Name: Partner ID, Length: 622, dtype: int64

In [195]:
display(dfOpp.head())
dfOpp.info()

,Partner ID,Opportunity Name,Forecasted Office Users,Forecasted Annual Revenue,CreateDateTime,Forecasted Close Date,Stage
0,65456,Opportunity 2905,79.0,74508.78,2020-08-19 11:12:00,2021-01-29,0. Downgraded
1,85139,Opportunity 2906,170.0,154128.12,2020-08-20 08:26:00,2020-12-31,0. Downgraded
2,63909,Opportunity 2907,800.0,87359.18,2020-05-29 05:24:00,2021-04-30,0. Downgraded
3,64261,Opportunity 2915,100.0,31.44,2020-06-16 03:25:00,2020-09-25,0. Downgraded
4,43258,Opportunity 2917,1500.0,165051.09,2020-09-07 02:53:00,2021-05-28,0. Downgraded


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2091 entries, 0 to 2090
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Partner ID                 2091 non-null   int64         
 1   Opportunity Name           2091 non-null   object        
 2   Forecasted Office Users    2085 non-null   float64       
 3   Forecasted Annual Revenue  2091 non-null   float64       
 4   CreateDateTime             2091 non-null   datetime64[ns]
 5   Forecasted Close Date      2091 non-null   datetime64[ns]
 6   Stage                      2091 non-null   object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 114.5+ KB


In [254]:
df = pd.merge(dfPartners, dfOpp, on='Partner ID')
df.set_index('CreateDateTime', inplace=True)
df.sort_values('CreateDateTime', axis = 0, inplace=True)
df['Opportunity'] = [1 for x in df['Opportunity Name']]
df['Closed Revenue'] = [df['Forecasted Annual Revenue'][i] if df['Stage'][i] == '7. Closed Won' else 0 for i in range(len(df))]
df['Lost Revenue'] = [df['Forecasted Annual Revenue'][i] if df['Stage'][i] == '0. Downgraded' else 0 for i in range(len(df))] 


columnsToBreak = ['Channel Manager', 'Partner Name','Stage' ]
breakColumns(df, columnsToBreak, conditionRevenue=columnsToBreak[:2])
df.head()

,Partner Name,Partner ID,Channel Manager,Opportunity Name,Forecasted Office Users,Forecasted Annual Revenue,Forecasted Close Date,Stage,Opportunity,Closed Revenue,Lost Revenue,Channel Manager 4,Channel Manager 4 Revenue,Channel Manager 4 Lost Revenue,Channel Manager 4 Closed Sale,Channel Manager 6,Channel Manager 6 Revenue,Channel Manager 6 Lost Revenue,Channel Manager 6 Closed Sale,Channel Manager 3,Channel Manager 3 Revenue,Channel Manager 3 Lost Revenue,Channel Manager 3 Closed Sale,Channel Manager 1,Channel Manager 1 Revenue,Channel Manager 1 Lost Revenue,Channel Manager 1 Closed Sale,Channel Manager 7,Channel Manager 7 Revenue,Channel Manager 7 Lost Revenue,Channel Manager 7 Closed Sale,Channel Manager 5,Channel Manager 5 Revenue,Channel Manager 5 Lost Revenue,Channel Manager 5 Closed Sale,Channel Manager 2,Channel Manager 2 Revenue,Channel Manager 2 Lost Revenue,Channel Manager 2 Closed Sale,Partner 506,...,Partner 502,Partner 502 Revenue,Partner 502 Lost Revenue,Partner 502 Closed Sale,Partner 402,Partner 402 Revenue,Partner 402 Lost Revenue,Partner 402 Closed Sale,Partner 421,Partner 421 Revenue,Partner 421 Lost Revenue,Partner 421 Closed Sale,Partner 410,Partner 410 Revenue,Partner 410 Lost Revenue,Partner 410 Closed Sale,Partner 458,Partner 458 Revenue,Partner 458 Lost Revenue,Partner 458 Closed Sale,Partner 84,Partner 84 Revenue,Partner 84 Lost Revenue,Partner 84 Closed Sale,Partner 298,Partner 298 Revenue,Partner 298 Lost Revenue,Partner 298 Closed Sale,Partner 219,Partner 219 Revenue,Partner 219 Lost Revenue,Partner 219 Closed Sale,0. Downgraded,7. Closed Won,1. Qualify,2. Problem,3. Solution,4. Proof,5. Agreement,6. Order
CreateDateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-03 14:23:00,Partner 269,30691,Channel Manager 3,Opportunity 1540,2.0,671.52,2020-01-08,7. Closed Won,1,671.52,0.00,0,0.0,0.0,0,0,0.0,0.0,0,1,671.52,0.0,1,0,0.0,0.0,0,0,0.0,0.00,0,0,0.0,0.0,0,0,0.0,0.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,1,0,0,0,0,0,0
2020-01-05 12:50:00,Partner 600,60082,Channel Manager 7,Opportunity 4424,1000.0,204191.77,2021-06-30,0. Downgraded,1,0.00,204191.77,0,0.0,0.0,0,0,0.0,0.0,0,0,0.00,0.0,0,0,0.0,0.0,0,1,0.0,204191.77,0,0,0.0,0.0,0,0,0.0,0.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,1,0,0,0,0,0,0,0
2020-01-05 12:51:00,Partner 600,60082,Channel Manager 7,Opportunity 4425,10000.0,2041917.74,2020-11-02,0. Downgraded,1,0.00,2041917.74,0,0.0,0.0,0,0,0.0,0.0,0,0,0.00,0.0,0,0,0.0,0.0,0,1,0.0,2041917.74,0,0,0.0,0.0,0,0,0.0,0.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,1,0,0,0,0,0,0,0
2020-01-05 12:52:00,Partner 600,60082,Channel Manager 7,Opportunity 4426,1000.0,0.00,2020-10-29,0. Downgraded,1,0.00,0.00,0,0.0,0.0,0,0,0.0,0.0,0,0,0.00,0.0,0,0,0.0,0.0,0,1,0.0,0.00,0,0,0.0,0.0,0,0,0.0,0.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,1,0,0,0,0,0,0,0
2020-01-05 12:56:00,Partner 600,60082,Channel Manager 7,Opportunity 4427,800.0,163353.42,2021-07-31,0. Downgraded,1,0.00,163353.42,0,0.0,0.0,0,0,0.0,0.0,0,0,0.00,0.0,0,0,0.0,0.0,0,1,0.0,163353.42,0,0,0.0,0.0,0,0,0.0,0.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0,0.0,0,0,0.0,0,0,1,0,0,0,0,0,0,0


In [227]:
partnerDict = {label: df[f'{label} Revenue'].sum() for label in df['Partner Name'].value_counts().index}
partners = pd.Series(partnerDict, )
topPartners = partners.sort_values(ascending = False)[:25]

In [255]:
df['Channel Manager 1 Closed Sale'].sum()

82

In [249]:
fig = go.Figure(
  data = [
    go.Bar(x = df['Partner Name'].value_counts()[:25].index, y = df['Partner Name'].value_counts()[:25]),
    go.Bar(x = topPartners.index, y = topPartners)
  ]
)
fig.show()

In [228]:
px.bar(x = topPartners.index, y = topPartners, )

In [177]:
dfMonth = df.resample('MS').sum()
dfMonth['Total Churn'] = 1 - round(dfMonth['7. Closed Won']/dfMonth['Opportunity'],2)
display((dfMonth.head()))
dfQuarter = df.resample('Q').sum()
dfQuarter['Total Churn'] = 1 - round(dfQuarter['7. Closed Won']/dfQuarter['Opportunity'],2)


,Partner ID,Forecasted Office Users,Forecasted Annual Revenue,Opportunity,Running Opportunities,Closed Revenue,Lost Revenue,Cumulative Forecasted Revenue,Cumulative Closed Revenue,Cumulative Lost Revenue,Channel Manager 4,Channel Manager 4 Revenue,Channel Manager 4 Lost Revenue,Channel Manager 6,Channel Manager 6 Revenue,Channel Manager 6 Lost Revenue,Channel Manager 3,Channel Manager 3 Revenue,Channel Manager 3 Lost Revenue,Channel Manager 1,Channel Manager 1 Revenue,Channel Manager 1 Lost Revenue,Channel Manager 7,Channel Manager 7 Revenue,Channel Manager 7 Lost Revenue,Channel Manager 5,Channel Manager 5 Revenue,Channel Manager 5 Lost Revenue,Channel Manager 2,Channel Manager 2 Revenue,Channel Manager 2 Lost Revenue,Partner 506,Partner 506 Revenue,Partner 506 Lost Revenue,Partner 562,Partner 562 Revenue,Partner 562 Lost Revenue,Partner 528,Partner 528 Revenue,Partner 528 Lost Revenue,...,Partner 481 Lost Revenue,Partner 385,Partner 385 Revenue,Partner 385 Lost Revenue,Partner 238,Partner 238 Revenue,Partner 238 Lost Revenue,Partner 502,Partner 502 Revenue,Partner 502 Lost Revenue,Partner 402,Partner 402 Revenue,Partner 402 Lost Revenue,Partner 421,Partner 421 Revenue,Partner 421 Lost Revenue,Partner 410,Partner 410 Revenue,Partner 410 Lost Revenue,Partner 458,Partner 458 Revenue,Partner 458 Lost Revenue,Partner 84,Partner 84 Revenue,Partner 84 Lost Revenue,Partner 298,Partner 298 Revenue,Partner 298 Lost Revenue,Partner 219,Partner 219 Revenue,Partner 219 Lost Revenue,0. Downgraded,7. Closed Won,1. Qualify,2. Problem,3. Solution,4. Proof,5. Agreement,6. Order,Total Churn
CreateDateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,5275597,50211.0,7830957.38,94,4465,27675.43,7639730.01,4.407861e+08,1600811.92,4.380699e+08,12,3095.28,1002681.27,3,0.00,54296.15,33,13568.50,743228.42,19,6130.85,554747.08,10,0.00,3043023.80,17,4880.80,2241753.29,0,0.00,0.00,4,0.00,423225.57,0,0.0,0.0,13,4880.80,2170035.14,...,0,0,0.00,0,1,0,23994.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,72,19,1,2,0,0,0,0,0.80
2020-02-01,5910729,28899.0,5705890.49,105,15435,68431.92,4886143.20,1.059132e+09,7697025.99,9.954921e+08,26,19737.73,991900.98,9,0.00,899721.94,21,8089.42,165015.62,23,18951.64,747235.73,3,0.00,241909.88,18,0.00,1775570.13,5,21653.13,64788.92,2,0.00,47126.02,0,0.0,0.0,6,0.00,1459192.35,...,0,0,0.00,0,0,0,0.0,0,0,0.0,1,0,1728.32,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,84,14,3,3,1,0,0,0,0.87
2020-03-01,8194289,29004.0,6316572.51,134,35711,266541.29,5764485.13,2.169905e+09,29582220.52,1.995623e+09,27,57790.41,931418.46,13,4729.58,121288.78,31,32130.46,491712.64,28,53122.87,813039.78,9,81539.78,397042.94,20,30099.04,1644661.39,6,7129.15,1365321.14,5,1696.88,138450.83,0,0.0,0.0,10,6834.69,267445.48,...,0,1,3106.74,0,0,0,0.0,1,0,754.2,0,0,0.00,0,0,0.0,1,0,502.52,0,0,0.0,1,0,1885.51,0,0,0.00,0,0.0,0,84,41,6,0,3,0,0,0,0.69
2020-04-01,5622912,19368.0,3003746.36,90,34065,94495.67,2403985.07,1.928371e+09,34276393.04,1.764165e+09,25,45690.38,633458.08,11,0.00,652090.90,26,17991.63,365662.15,11,1963.95,69386.22,8,24576.62,103754.33,7,3518.89,501278.14,2,754.20,78355.25,7,4529.47,396708.94,1,0.0,137401.1,5,0.00,281320.06,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,1,0,1871.04,0,0.0,0,58,21,5,4,1,0,1,0,0.77
2020-05-01,6156547,23696.0,5817174.95,101,47874,504838.11,5144969.14,2.675433e+09,74601544.75,2.420458e+09,16,68130.37,1241486.20,13,2765.16,1327092.16,19,30800.00,171011.95,29,48663.94,2123887.62,13,53643.04,103560.64,8,222669.56,173209.17,3,78166.04,4721.40,4,58887.87,216650.78,0,0.0,0.0,5,162914.23,173209.17,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,56,37,3,5,0,0,0,0,0.63


In [190]:
top10 = list(df['Partner Name'].value_counts()[:10].index)


In [259]:
import plotly.express as px
import plotly.graph_objects as go

# px.line(data_frame=dfQuarter, x = dfQuarter.index, y ='Closed Revenue',)
# px.line(data_frame=dfMonth, x = dfMonth.index, y = 'Closed Revenue')
# px.bar(data_frame = dfMonth, x = dfMonth.index, y = dfMonth['Channel Manager 6'], barmode='stack')
# px.line(data_frame=dfMonth, x = dfMonth.index, y ='Forecasted Annual Revenue', color = 'Channel Manager 1')

fig = go.Figure(
  data = [
    go.Bar(
      name = label, 
      x = dfMonth.index, 
      y = dfMonth[label],
      text = dfMonth[label],
      textposition = 'inside',
      textangle = 0,

    ) for label in sorted(dfPartners['Channel Manager'].unique())
  ],

  layout = {
      'title': 'Opportunities per Month by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Month',
          'tickangle' : -90,
          'nticks' : 16
      },
      'yaxis' : {
          'title' : 'Opportunity Count',
      }
  }
)
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = dfQuarter[label],
      text = dfQuarter[label],
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Channel Manager'].unique())] + \
  [
  go.Scatter(
    name = 'Total Opportunities',
    x = dfQuarter.index, 
    y = dfQuarter['Opportunity'], 
    text = dfQuarter['Opportunity'],
    mode="lines+text+markers",
    textposition = 'top left',
    line = {'color' : 'black'}
  )
  ],
     layout = {
      'title': 'Opportunities per Quarter by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          'nticks' : 6,
          
      },
      'yaxis' : {
          'title' : 'Opportunity Count',
      }
  }
)
# Change the bar mode
fig3.update_layout(barmode='stack')
fig3.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = round(dfQuarter[f'{label} Revenue'],0),
      text = round(dfQuarter[f'{label} Revenue'],0),
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Channel Manager'].unique())] + \
  [
  go.Scatter(
    name = 'Total Closed Revenue',
    x = dfQuarter.index, 
    y = round(dfQuarter['Closed Revenue'],2), 
    text = round(dfQuarter['Closed Revenue'],0),
    mode="lines+text+markers",
    textposition = 'top left',
    line = {'color' : 'black'}
  )
  ],
     layout = {
      'title': 'Closed Revenue per Quarter by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          'nticks' : 6,
          
      },
      'yaxis' : {
          'title' : 'Revenue',
      }
  }
)
# Change the bar mode
fig3.update_layout(barmode='stack')
fig3.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = round(dfQuarter[f'{label} Lost Revenue'],0),
      text = round(dfQuarter[f'{label} Lost Revenue'],0),
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Channel Manager'].unique())] + \
  [
  go.Scatter(
    name = 'Total Lost Revenue',
    x = dfQuarter.index, 
    y = round(dfQuarter['Lost Revenue'],0), 
    text = round(dfQuarter['Lost Revenue'],0),
    mode="lines+text+markers",
    textposition = 'top left',
    line = {'color' : 'red'}
  )
  ],
     layout = {
      'title': 'Lost Revenue per Quarter by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          'nticks' : 6,
          
      },
      'yaxis' : {
          'title' : 'Revenue',
      }
  }
)
# Change the bar mode
fig3.update_layout(barmode='stack')
fig3.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = round(dfQuarter[f'{label} Revenue'],0),
      text = round(dfQuarter[f'{label} Revenue'],0),
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Partner Name'].unique()) if label in top10] + \
  [
  go.Scatter(
    name = 'Total Lost Revenue',
    x = dfQuarter.index, 
    y = round(dfQuarter['Closed Revenue'],0), 
    text = round(dfQuarter['Closed Revenue'],0),
    mode="lines+text+markers",
    textposition = 'top left',
    line = {'color' : 'black'}
  )
  ],
     layout = {
      'title': 'Revenue per Quarter by Partner',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          'nticks' : 6,
          
      },
      'yaxis' : {
          'title' : 'Revenue',
      }
  }
)
# Change the bar mode
fig3.update_layout(barmode='stack')
fig3.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = round(dfQuarter[f'{label} Lost Revenue'],0),
      text = round(dfQuarter[f'{label} Lost Revenue'],0),
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Partner Name'].unique()) if label in top10] + \
  [
  go.Scatter(
    name = 'Total Lost Revenue',
    x = dfQuarter.index, 
    y = round(dfQuarter['Lost Revenue'],0), 
    text = round(dfQuarter['Lost Revenue'],0),
    mode="lines+text+markers",
    textposition = 'top left',
    line = {'color' : 'red'}
  )
  ],
     layout = {
      'title': 'Lost Revenue per Quarter by Partner',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          'nticks' : 6,
          
      },
      'yaxis' : {
          'title' : 'Revenue',
      }
  }
)
# Change the bar mode
fig3.update_layout(barmode='stack')
fig3.show()

fig3 = go.Figure(data=[
  go.Bar(
      name = label, 
      x = dfQuarter.index, 
      y = round(dfQuarter[f'{label} Revenue']*dfQuarter[label]/dfQuarter['Opportunity'],0),
      text = round(dfQuarter[f'{label} Revenue'],0),
      textposition = 'inside',
      textangle = 0,
    ) for label in sorted(dfPartners['Partner Name'].unique()) if label in top10] + \
  [
  # go.Scatter(
  #   name = 'Total Lost Revenue',
  #   x = dfQuarter.index, 
  #   y = round(dfQuarter['Lost Revenue'],0), 
  #   text = round(dfQuarter['Lost Revenue'],0),
  #   mode="lines+text+markers",
  #   textposition = 'top left',
  #   line = {'color' : 'red'}
  # )
  ],
     layout = {
      'title': 'Lost Revenue per Quarter by Partner',
      'xaxis' : {
          'title' : 'Reporting Quarter',
          'tickangle': -90,
          'nticks' : 6,
          
      },
      'yaxis' : {
          'title' : 'Revenue',
      }
  }
)
# Change the bar mode
fig3.update_layout(barmode='group')
fig3.show()

In [200]:
fig2 = go.Figure(data=[
    go.Scatter(x = dfQuarter.index, y = dfQuarter['Closed Revenue']),
    go.Scatter(x = dfQuarter.index, y = dfQuarter.diff(1)['Closed Revenue']),
])
fig2.show()

fig4 = go.Figure(data=[
    go.Scatter(x = dfQuarter.index, y = dfQuarter['Total Churn']),
    go.Scatter(x = dfQuarter.index, y = dfQuarter.diff(1)['Total Churn']),
])
fig4.show()

fig5 = go.Figure(
  data = [
    go.Scatter(
      name = label, 
      x = dfQuarter.index, 
      y = dfQuarter[f'{label} Revenue'],
      text = round(dfQuarter[f'{label} Revenue'],0),
    ) for label in sorted(dfPartners['Channel Manager'].unique())
  ] + \
  [go.Bar(name = label, x = dfQuarter.index, y = dfQuarter.diff(1)[f'{label} Revenue']) for label in sorted(dfPartners['Channel Manager'].unique())],

  layout = {
      'title': 'Opportunities per Month by Channel Manager',
      'xaxis' : {
          'title' : 'Reporting Month',
          'tickangle' : -90,
          'nticks' : 16
      },
      'yaxis' : {
          'title' : 'Opportunity Count',
      }
  }
)
fig5.show()

In [182]:
fig2 = go.Figure(data=[
    go.Scatter(name = "Closed Revenue", x = dfMonth.index, y = dfMonth['Closed Revenue']),
    go.Bar(name = 'Opportunities Per Month',x = dfMonth.index, y = dfMonth['Opportunity']*10000),
    go.Scatter(name = 'Lost Revenue',x = dfMonth.index, y = dfMonth['Lost Revenue']),
])
fig2.show()

In [183]:
fig2 = go.Figure(data=[
    go.Scatter( x = dfQuarter.index, y = dfQuarter['Closed Revenue']),
    go.Bar(x = dfQuarter.index, y = dfQuarter['Opportunity']*10000),
    go.Scatter(x = dfQuarter.index, y = dfQuarter['Lost Revenue']),
])
fig2.show()

In [235]:
dfMonth.head()

,Partner ID,Forecasted Office Users,Forecasted Annual Revenue,Opportunity,Running Opportunities,Closed Revenue,Lost Revenue,Cumulative Forecasted Revenue,Cumulative Closed Revenue,Cumulative Lost Revenue,Channel Manager 4,Channel Manager 4 Revenue,Channel Manager 4 Lost Revenue,Channel Manager 6,Channel Manager 6 Revenue,Channel Manager 6 Lost Revenue,Channel Manager 3,Channel Manager 3 Revenue,Channel Manager 3 Lost Revenue,Channel Manager 1,Channel Manager 1 Revenue,Channel Manager 1 Lost Revenue,Channel Manager 7,Channel Manager 7 Revenue,Channel Manager 7 Lost Revenue,Channel Manager 5,Channel Manager 5 Revenue,Channel Manager 5 Lost Revenue,Channel Manager 2,Channel Manager 2 Revenue,Channel Manager 2 Lost Revenue,Partner 506,Partner 506 Revenue,Partner 506 Lost Revenue,Partner 562,Partner 562 Revenue,Partner 562 Lost Revenue,Partner 528,Partner 528 Revenue,Partner 528 Lost Revenue,...,Partner 481 Lost Revenue,Partner 385,Partner 385 Revenue,Partner 385 Lost Revenue,Partner 238,Partner 238 Revenue,Partner 238 Lost Revenue,Partner 502,Partner 502 Revenue,Partner 502 Lost Revenue,Partner 402,Partner 402 Revenue,Partner 402 Lost Revenue,Partner 421,Partner 421 Revenue,Partner 421 Lost Revenue,Partner 410,Partner 410 Revenue,Partner 410 Lost Revenue,Partner 458,Partner 458 Revenue,Partner 458 Lost Revenue,Partner 84,Partner 84 Revenue,Partner 84 Lost Revenue,Partner 298,Partner 298 Revenue,Partner 298 Lost Revenue,Partner 219,Partner 219 Revenue,Partner 219 Lost Revenue,0. Downgraded,7. Closed Won,1. Qualify,2. Problem,3. Solution,4. Proof,5. Agreement,6. Order,Total Churn
CreateDateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,5275597,50211.0,7830957.38,94,4465,27675.43,7639730.01,4.407861e+08,1600811.92,4.380699e+08,12,3095.28,1002681.27,3,0.00,54296.15,33,13568.50,743228.42,19,6130.85,554747.08,10,0.00,3043023.80,17,4880.80,2241753.29,0,0.00,0.00,4,0.00,423225.57,0,0.0,0.0,13,4880.80,2170035.14,...,0,0,0.00,0,1,0,23994.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,72,19,1,2,0,0,0,0,0.80
2020-02-01,5910729,28899.0,5705890.49,105,15435,68431.92,4886143.20,1.059132e+09,7697025.99,9.954921e+08,26,19737.73,991900.98,9,0.00,899721.94,21,8089.42,165015.62,23,18951.64,747235.73,3,0.00,241909.88,18,0.00,1775570.13,5,21653.13,64788.92,2,0.00,47126.02,0,0.0,0.0,6,0.00,1459192.35,...,0,0,0.00,0,0,0,0.0,0,0,0.0,1,0,1728.32,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,84,14,3,3,1,0,0,0,0.87
2020-03-01,8194289,29004.0,6316572.51,134,35711,266541.29,5764485.13,2.169905e+09,29582220.52,1.995623e+09,27,57790.41,931418.46,13,4729.58,121288.78,31,32130.46,491712.64,28,53122.87,813039.78,9,81539.78,397042.94,20,30099.04,1644661.39,6,7129.15,1365321.14,5,1696.88,138450.83,0,0.0,0.0,10,6834.69,267445.48,...,0,1,3106.74,0,0,0,0.0,1,0,754.2,0,0,0.00,0,0,0.0,1,0,502.52,0,0,0.0,1,0,1885.51,0,0,0.00,0,0.0,0,84,41,6,0,3,0,0,0,0.69
2020-04-01,5622912,19368.0,3003746.36,90,34065,94495.67,2403985.07,1.928371e+09,34276393.04,1.764165e+09,25,45690.38,633458.08,11,0.00,652090.90,26,17991.63,365662.15,11,1963.95,69386.22,8,24576.62,103754.33,7,3518.89,501278.14,2,754.20,78355.25,7,4529.47,396708.94,1,0.0,137401.1,5,0.00,281320.06,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,1,0,1871.04,0,0.0,0,58,21,5,4,1,0,1,0,0.77
2020-05-01,6156547,23696.0,5817174.95,101,47874,504838.11,5144969.14,2.675433e+09,74601544.75,2.420458e+09,16,68130.37,1241486.20,13,2765.16,1327092.16,19,30800.00,171011.95,29,48663.94,2123887.62,13,53643.04,103560.64,8,222669.56,173209.17,3,78166.04,4721.40,4,58887.87,216650.78,0,0.0,0.0,5,162914.23,173209.17,...,0,0,0.00,0,0,0,0.0,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.0,0,0,0.00,0,0,0.00,0,0.0,0,56,37,3,5,0,0,0,0,0.63


In [237]:
fig2 = go.Figure(data=[
    go.Scatter( x = dfMonth[:'03-01-2021'].index, y = dfMonth['Forecasted Annual Revenue'].diff(1)/dfMonth['Forecasted Annual Revenue']),
    go.Scatter( x = dfMonth[:'03-01-2021'].index, y = dfMonth['Closed Revenue'].diff(1)/dfMonth['Closed Revenue'], line = {'color':'green'}),
    # go.Bar(x = dfQuarter.index, y = dfQuarter['Opportunity']*10000),
    go.Scatter(x = dfQuarter.index, y = [0 for x in dfQuarter.index]),
])
fig2.show()

In [244]:
fig2 = go.Figure(data=[
    go.Scatter( x = dfQuarter[:'03-31-2021'].index, y = dfQuarter['Closed Revenue'].diff(1)/dfQuarter['Closed Revenue']*100),
    go.Scatter( x = dfQuarter[:'03-31-2021'].index, y = dfQuarter['Forecasted Annual Revenue'].diff(1)/dfQuarter['Forecasted Annual Revenue']*100),

    # go.Scatter( x = dfMonth[:'03-31-2021'].index, y = dfMonth['Closed Revenue'].diff(1)/dfMonth['Closed Revenue']),
    # go.Bar(x = dfQuarter.index, y = dfQuarter['Opportunity']*10000),
    # go.Scatter(x = dfQuarter.index, y = dfQuarter['Lost Revenue']),
])
fig2.show()